<a href="https://colab.research.google.com/github/notprameghuikey0913/Sentiment-Analaysis-with-Deep-Learning-using-BERT/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis with Deep Learning using BERT

### Prerequisites

- Intermediate-level knowledge of Python 3 (NumPy and Pandas preferably, but not required)
- Exposure to PyTorch usage
- Basic understanding of Deep Learning and Language Models (BERT specifically)

### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

### What is BERT

BERT is a large-scale transformer-based Language Model that can be finetuned for a variety of tasks.

For more information, the original paper can be found [here](https://arxiv.org/abs/1810.04805). 

[HuggingFace documentation](https://huggingface.co/transformers/model_doc/bert.html)

[Bert documentation](https://characters.fandom.com/wiki/Bert_(Sesame_Street) ;)

<img src="Images/BERT_diagrams.pdf" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [2]:
import torch
!pip install transformers
import pandas as pd
from tqdm.notebook import tqdm

     |████████████████████████████████| 3.8 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
     |████████████████████████████████| 596 kB 41.7 MB/s 
     |████████████████████████████████| 6.5 MB 30.1 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
df = pd.read_csv("/smile-annotations-final.csv", 
                 names = ['id', 'text', 'category'])
df.set_index('id', inplace = True)

In [4]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [5]:
# counts up how many times each unique instance occurs inside the data
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [6]:
#remove categories that have multiple emotions
# | is a reserved symbol in python so you have to put an escape character before it \
df = df[~df.category.str.contains('\|')]

In [7]:
df = df[df.category != 'nocode']

In [8]:
df.category.value_counts()
# we have a class imbalance here

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [9]:
possible_labels = df.category.unique()

In [10]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [11]:
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [12]:
#df['labels'] = df.category.replace(label_dict)
df['labels'] = df.category.apply(lambda x: label_dict[x])
df.head()

,text,category,labels
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


## Task 3: Training/Validation Split

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    df.index.values,
    df.labels.values,
    test_size = 0.15,
    random_state = 17,
    stratify = df.labels.values
)

In [15]:
#create a column for each instance to mention whether it is in the train or test set
df["data_type"] = ['not_set']*df.shape[0]

In [16]:
#if instance exists in X_train, set 'data_type' to 'train'
#if intance exists in X_test, set 'data_type' to 'test'
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_test, 'data_type'] = 'test'

In [17]:
#check if what we wanted to happen has happened by grouping the three
#and getting a count of each sub group
df.groupby(['category', 'labels', 'data_type']).count()

text
category     labels data_type      
angry        2      test          9
                    train        48
disgust      3      test          1
                    train         5
happy        0      test        171
                    train       966
not-relevant 1      test         32
                    train       182
sad          4      test          5
                    train        27
surprise     5      test          5
                    train        30

## Task 4: Loading Tokenizer and Encoding our Data

In [18]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [19]:
#convert word to numbers rep.

tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased', # using all lowercase data
    do_lower_case = True # convert everything to lower case because we're using 
                         # uncased model of BERT 
)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [20]:
# convert all sentences from tweets from language to encoded form (tokens)
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens = True, #BERT's way of knowing whena sent ends and a new one begins
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensor = 'pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    df[df.data_type == 'test'].text.values,
    add_special_tokens = True, #BERT's way of knowing when a sen ends and a new one begins
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensor = 'pt'
)

#encoded_data_train returns a dictionary

#input ids basically represent each word as a number
input_ids_train = torch.tensor(encoded_data_train['input_ids'])
attention_masks_train = torch.tensor(encoded_data_train['attention_mask'])
labels_train = torch.tensor(df[df.data_type == 'train'].labels.values)

input_ids_test = torch.tensor(encoded_data_test['input_ids'])
attention_masks_test = torch.tensor(encoded_data_test['attention_mask'])
labels_test = torch.tensor(df[df.data_type == 'test'].labels.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Keyword arguments {'return_tensor': 'pt'} not recognized.
Keyword arguments {'return_tensor': 'pt'} 

In [21]:
# creating train and test datasets of tensor values of the encoded data from 
# the previous cell

dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train, 
                              labels_train)

dataset_test = TensorDataset(input_ids_test, 
                             attention_masks_test, 
                             labels_test)

In [22]:
len(dataset_train)

1258

In [23]:
len(dataset_test)

223

## Task 5: Setting up BERT Pretrained Model

In [24]:
from transformers import BertForSequenceClassification

In [25]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Task 6: Creating Data Loaders

In [26]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [27]:
# offer a way to iterate through your dataset in batches
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = batch_size
)

dataloader_val = DataLoader(
    dataset_test,
    sampler = RandomSampler(dataset_test),
    batch_size = 32
)

## Task 7: Setting Up Optimizer and Scheduler

In [28]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [29]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, # 2e-5 5e-5
    eps = 1e-8
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [30]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [31]:
import numpy as np

In [32]:
from sklearn.metrics import f1_score

In [33]:
#predss = [0.9 0.05 0.05 0 0 0]
#preds = [1 0 0 0 0 0]

In [34]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis =1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [35]:
def accuracy_per_class(preds, labels):
    labels_dict_inverse = {v: k for k, v in  label_dict.items()}
    
    preds_flat = np.argmax(preds, axis =1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f'Class: {labels_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [36]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [37]:
# checkwhat device you have and set it properly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device) # transfer our model to the device
print(device)

cuda


In [38]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [42]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train() # send model to training mode
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                       desc = 'Epoch {:1d}'.format(epoch),
                       leave = False, # let it override itself after each epoch
                       disable = False
                       ) # how many batches have been trained and how many to go
    for batch in progress_bar:
        
        model.zero_grad() # initialize grads to 0
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            'labels':         batch[2]
            
        }
        
        outputs = model(**inputs) # unpacks the dictionary straight into the input
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward() #backprop of bert
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
        
    #torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_values = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_values)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.14043262844927432


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.6924118761505399
F1 Score (weighted): 0.8637792474498882


...

Validation loss: 0.7508187464305333
F1 Score (weighted): 0.879699349972373


Epoch 10:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.018007412892378245


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.7532966392380851
F1 Score (weighted): 0.879699349972373


## Task 10: Loading and Evaluating our Model

In [43]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [47]:
model.to(device)
print(device)

cuda


In [ ]:
model.load_state_dict(
    torch.load('Models/finetuned_bert_epoch_1_gpu_trained.model',
    map_location = torch.device('cpu'))
)

<All keys matched successfully>

In [50]:
_, predictions, true_vals = evaluate(dataloader_train)

  0%|          | 0/315 [00:00<?, ?it/s]

In [51]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 0/966
Class: not-relevant
Accuracy: 0/182
Class: angry
Accuracy: 0/48
Class: disgust
Accuracy: 5/5
Class: sad
Accuracy: 0/27
Class: surprise
Accuracy: 0/30


In [ ]:
# Google Colab -- GPU Instance (K80)
# batch_size = 32
# epoch = 10